In [1]:
import os
os.chdir('C:\\Users\\HP\\OneDrive\\Desktop\\Akaike\\Basics')
os.getcwd()

'C:\\Users\\HP\\OneDrive\\Desktop\\Akaike\\Basics'

In [2]:
from sqlalchemy import create_engine,text
import pandas as pd
import json
from dotenv import load_dotenv
from langchain_community.chat_models import  ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import HumanMessage, SystemMessage
load_dotenv(".env")

True

In [3]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import BM25Retriever,EnsembleRetriever
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [4]:
username = 'user_demo'
password = 'password_demo'
host = 'chat-csv.c9pssgmnnguu.us-west-2.rds.amazonaws.com'
port = 3306
database = 'sql_test_datasets'

In [5]:
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

In [6]:
query = 'SELECT * FROM supermarket_b'

In [7]:
df = pd.read_sql(text(query), engine)

In [8]:
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,None,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [9]:

def get_table_names(engine):
    query = """
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'sql_test_datasets'
    """
    table_names_df = pd.read_sql(text(query), engine)
    return table_names_df['TABLE_NAME'].tolist()

In [10]:
 get_table_names(engine)

['002_hul',
 'bike_rentals',
 'byob_product',
 'byob_sales',
 'byob_user',
 'column_anomalies',
 'dress',
 'hr_dataset',
 'jee_dataset',
 'myntra',
 'ps4',
 'stock_prices_hindunilvr',
 'stock_prices_maruti',
 'supermarket_b',
 'titanic']

In [11]:
import pandas as pd
from sqlalchemy import create_engine, text


def get_table_metadata(engine, table_names):
    res = []
    for table_name in table_names:
        query = f"""
        SELECT table_name, COLUMN_NAME, DATA_TYPE 
        FROM information_schema.columns
        WHERE table_name = '{table_name}' AND table_schema = 'sql_test_datasets'
        """
        res.append(pd.read_sql(text(query), engine))
    
    return pd.concat(res, ignore_index=True)





In [12]:
table_names = get_table_names(engine)
# table_names=['byob_sales','byob_user','byob_product']
metadata = get_table_metadata(engine, table_names)
metadata

,TABLE_NAME,COLUMN_NAME,DATA_TYPE
0,002_hul,Date,text
1,002_hul,Open,double
2,002_hul,High,double
3,002_hul,Low,double
4,002_hul,Close,double
...,...,...,...
174,titanic,Parch,bigint
175,titanic,Ticket,text
176,titanic,Fare,double
177,titanic,Cabin,text


In [13]:
meta =metadata.values.tolist()

In [14]:
meta = [str(i) for i in meta]

In [15]:
meta 

["['002_hul', 'Date', 'text']",
 "['002_hul', 'Open', 'double']",
 "['002_hul', 'High', 'double']",
 "['002_hul', 'Low', 'double']",
 "['002_hul', 'Close', 'double']",
 "['002_hul', 'Adj Close', 'double']",
 "['002_hul', 'Volume', 'bigint']",
 "['bike_rentals', 'instant', 'bigint']",
 "['bike_rentals', 'dteday', 'text']",
 "['bike_rentals', 'season', 'bigint']",
 "['bike_rentals', 'yr', 'bigint']",
 "['bike_rentals', 'mnth', 'bigint']",
 "['bike_rentals', 'holiday', 'bigint']",
 "['bike_rentals', 'weekday', 'bigint']",
 "['bike_rentals', 'workingday', 'bigint']",
 "['bike_rentals', 'weathersit', 'bigint']",
 "['bike_rentals', 'temp', 'double']",
 "['bike_rentals', 'atemp', 'double']",
 "['bike_rentals', 'hum', 'double']",
 "['bike_rentals', 'windspeed', 'double']",
 "['bike_rentals', 'casual', 'bigint']",
 "['bike_rentals', 'registered', 'bigint']",
 "['bike_rentals', 'cnt', 'bigint']",
 "['byob_product', 'Product ID', 'text']",
 "['byob_product', 'Product Name', 'text']",
 "['byob_pro

In [16]:
vectordb = FAISS.from_texts(meta,embedding=OpenAIEmbeddings(model="text-embedding-3-large"))

C:\Users\HP\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [17]:
def retriever1(query, k=3):
    results = vectordb.similarity_search(query, k=k)
    # print(results)
    contexts = [result.page_content for result in results]
    return contexts

In [25]:
# def retriever(query, k=4):
#     results = vectordb.similarity_search_with_relevance_scores(query, k=k)
#     print(results)
#     if not results:
#         return []
#     documents = [result[0] for result in results]
#     relevance_scores = [result[1] for result in results]
#     sorted_results = sorted(zip(documents, relevance_scores), key=lambda x: x[1], reverse=True)
#     print(sorted_results)
#     contexts = [doc.page_content for doc, score in sorted_results]
    
#     return contexts


In [19]:
question = "Yearly Shipping Cost Trend Over Time"
contexts = retriever1(question,5)
contexts

["['byob_sales', 'Shipping_Cost', 'double']",
 "['byob_sales', 'Profit', 'double']",
 "['hr_dataset', 'MonthlyRate', 'bigint']",
 "['hr_dataset', 'OverTime', 'text']",
 "['byob_sales', 'Order_Date', 'text']"]

In [26]:
# question = "Yearly Shipping Cost Trend Over Time"
# contexts = retriever(question,7)
# contexts

In [27]:
def column_retrieval(query, contexts):
    prompt_template = '''
    Given Information:
    Query: `{query}`
    Context: `{contexts}`
    **Important Instructions:**

    - Break down the question to understand its requirements.
    - From the `Context` retrieve only related columns from all the tables
    - Identify specific columns necessary to answer the query from the contexts.
    - Use the `context` and retrive the relavent columns.
    - Format your response as a JSON structure.
    
    - In `contexts`, the first column represents the table name.
     
    For Example:
       Contexts = ["['supermarket_b', 'Outlet_Type', 'text']", "['myntra', 'ProductBrand', 'text']"]
        - 'supermarket_b' and 'myntra' are table names,
        - 'Outlet_Type' and 'ProductBrand' are column names,
        - 'text' is the data type.
    - Use contexts and give response only relevant tables and columns 
    
    You need to give in JSON Output Result Format 
      
    - JSON Output Result Format:
    
    {{
        "TABLENAMEA": [["COLUMNNAMEA", "DATATYPEA"], ["COLUMNNAMEB", "DATATYPEB"]],
        "TABLENAMEB": [["COLUMNNAMEA", "DATATYPEA"], ["COLUMNNAMEB", "DATATYPEB"]]
    }}

    Examples:
    
    `Query`: "Average Sales by Product Name"
    `Contexts`:["['byob_sales', 'Product ID', 'text']","['byob_sales', 'Sales', 'double']", "['byob_sales', 'Profit', 'double']", "['byob_sales', 'Quantity', 'double']", "['byob_product', 'Product Name', 'text']", "['byob_sales', 'Shipping_Cost', 'double']","['byob_sales', 'Order_Date', 'text']"]
   
    Response:
    {{
        "byob_sales": [["Sales", "double"]],
        "byob_product": [["Product Name", "text"]]
    }}
     
    `Query`: "Profit Trend Over Time"
    `Contexts`:["['byob_sales', 'Profit', 'double']","['byob_sales', 'Time', 'text']","['byob_sales', 'Sales', 'double']","['byob_sales', 'Order_Date', 'text']","['byob_sales', 'Discount', 'double']","['byob_sales', 'Shipping_Cost', 'double']","['byob_sales', 'Order_Priority', 'text']"]
    
    Response:
    {{
        "byob_sales": [["Profit", "double"],["Order_Date", "text"]]
    }}


    query: {query}
    context: `{contexts}`
    Response:
    '''
    return prompt_template.format(query=query, contexts=contexts)


In [28]:
def model_response(query:str):
    chat = ChatOpenAI(model="gpt-3.5-turbo",
                    temperature = 0,
                    model_kwargs= {
                    "response_format": {"type": "json_object"},
                    "seed" : 42})

    messages = [
        SystemMessage(
            content="You are a business analyst designed to give key insights,designed to output json"),
        HumanMessage(
            content=query),
    ]
    return json.loads(chat(messages).content)

In [29]:

openai_api_key = os.getenv('OPENAI_API_KEY')
  

def rag_flow(question, retriever, k=7):
    contexts = retriever1(question, k)
    response =  model_response(column_retrieval(query, contexts))
    return response


In [30]:
question = "Shipping Cost by Product Name"
response = rag_flow(question, retriever)
response


{'byob_product': [['Product Name', 'text'],
  ['Product ID', 'text'],
  ['Category', 'text']],
 'byob_sales': [['Shipping_Cost', 'double'], ['Product ID', 'text']]}